In [ ]:
#导入需要的包
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import Dense
from keras.layers import Activation
from keras.layers import Dropout
from keras.layers.pooling import GlobalAveragePooling2D

#参数
height=224
width=224
batch_size = 32

#读入数据
datagen = ImageDataGenerator(
    rotation_range = 30,     # 随机旋转度数
    #width_shift_range = 0.2, # 随机水平平移
    #height_shift_range = 0.2,# 随机竖直平移
    #rescale = 1/255,         # 数据归一化
    #shear_range = 0.2,       # 随机裁剪
    #zoom_range = 0.2,        # 随机放大
    horizontal_flip = True,  # 随机水平翻转
    vertical_flip = True,   #随机竖直翻转
    #fill_mode = 'nearest',   # 填充方式
) 
test_datagen = ImageDataGenerator(
    rotation_range = 20,     # 随机旋转度数
    horizontal_flip = True,  # 随机水平翻转
    vertical_flip = True,   #随机竖直翻转
)

# 生成训练数据
train_generator = train_datagen.flow_from_directory(
    'image/train',
    target_size=(height,width),
    batch_size=batch_size,
    )

# 测试数据
test_generator = test_datagen.flow_from_directory(
    'image/test',
    target_size=(height,width),
    batch_size=batch_size,
    )


#搭建densenet网络
base_model = DenseNet169(include_top=False, weights='imagenet', input_shape=(height,width, 3))
x = base_model.output
x = Dropout(0.5)(x)
x = GlobalAveragePooling2D()(x) 
predictions = Dense(12, activation='softmax')(x) #new softmax layer
model = Model(input=base_model.input, output=predictions)
model.summary()
#编译和训练网络
adam = Adam(lr=1e-4)
model.compile(optimizer=adam,loss='categorical_crossentropy',metrics=['accuracy'])
model.fit_generator(train_generator,epochs=30,validation_data=test_generator)

#保存模型
model.save('model_cnn.h5')